In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/shubham/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_df=pd.read_csv("train_2kmZucJ.csv")

In [3]:
train_df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
test_df=pd.read_csv("test_oJQbWVk.csv")

In [5]:
test_df.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [6]:
import nltk

In [7]:
tokenizer = nltk.tokenize.TreebankWordTokenizer()

In [8]:
train_cp = train_df.copy()
test_cp = test_df.copy()

In [9]:
train_df = train_df.drop("id", axis=1)

In [10]:
train_df.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...
2,0,We love this! Would you go? #talk #makememorie...
3,0,I'm wired I know I'm George I was made that wa...
4,1,What amazing service! Apple won't even talk to...


In [11]:
train_df.describe()

,label
count,7920.000000
mean,0.255808
std,0.436342
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [12]:
for index,tweet in enumerate(train_df["tweet"][10:15]):
    print(index+1,".",tweet)
for index,tweet in enumerate(test_df["tweet"][10:15]):
    print(index+1,".",tweet)

1 . hey #apple when you make a new ipod dont make it a new color or .2inches thinner make it not crash every five fuckin minite.
2 . Ha! Not heavy machinery but it does what I need it to. @Apple really dropped the ball with that design. #drinkyourhaterade
3 . Contemplating giving in to the iPhone bandwagon simply because #Cellcom has no new Androids #depressing #IDontWantTo
4 . I just made another crazy purchase lol my theory is 'Work hard, Play Hard' Lol #iPad #Apple #Shopping http://instagr.am/p/NUyFHvyE7I/
5 . @shaqlockholmes @sam_louise1991 the battery is so painful! I charge it overnight and by lunchtime battery is dead! #HateOrange.
1 . Con mi buddy#edgar #buddy #friend #viviendo #taller #friendly #hug #sony #smile #sonrisa http://instagram.com/p/fasVZruzQI/
2 . #MyMomIsGreatBecause she always care about me! And do anything for me #FOLLOW #MothersDay #MustFollow mothersday #iPhone #tweet
3 . #kiss,,#romance,#hot,#movie,#iphone,#sex,#porn Lustful Mature Woman Is Getting Nailed Int

In [13]:
import re
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [14]:
train_df["clean_text"] = train_df.tweet.apply(lambda x: clean_text(x))
test_df["clean_text"] = test_df.tweet.apply(lambda x: clean_text(x))

In [15]:
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

In [16]:
def gen_freq(text):
    word_list = []
    
    for tw_words in text.split():
        word_list.extend(tw_words)
        
    word_freq = pd.Series(word_list).value_counts()
    
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
        else:
            return 0
        
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0

def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
        else:
            return 0



In [18]:
word_freq_train = gen_freq(train_df.clean_text.str)
rare_100 = word_freq_train[-100]
train_df.head()
word_freq_test = gen_freq(test_df.clean_text.str)
test_df.head()

,id,tweet,clean_text
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",i'd like to puts some cd roms on my ipad is t...
3,7924,My ipod is officially dead. I lost all my pict...,my ipod is officially dead i lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...,been fighting itunes all night i only want th...


In [20]:
train_df['word_count'] = train_df.clean_text.str.split().apply(lambda x: len(x))
test_df['word_count'] = test_df.clean_text.str.split().apply(lambda x: len(x))

train_df['any_neg'] = train_df.clean_text.str.split().apply(lambda x: any_neg(x))
test_df['any_neg'] = test_df.clean_text.str.split().apply(lambda x: any_neg(x))

train_df['is_question'] = train_df.clean_text.str.split().apply(lambda x: is_question(x))
test_df['is_question'] = test_df.clean_text.str.split().apply(lambda x: is_question(x))

train_df['char_count'] = train_df.clean_text.apply(lambda x: len(x))
test_df['char_count'] = test_df.clean_text.apply(lambda x: len(x))

In [21]:
gen_freq(train_df.clean_text.str)[:10]

iphone       4313
apple        2974
p            2784
instagram    2187
samsung      1441
twitter      1262
new          1160
https        1014
phone         998
sony          855
dtype: int64

In [22]:
test_df.head()

,id,tweet,clean_text,word_count,any_neg,is_question,char_count
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade won't let me d...,14,0,0,77
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple i...,16,0,0,115
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",i'd like to puts some cd roms on my ipad is t...,21,0,0,104
3,7924,My ipod is officially dead. I lost all my pict...,my ipod is officially dead i lost all my pict...,24,0,0,129
4,7925,Been fighting iTunes all night! I only want th...,been fighting itunes all night i only want th...,13,0,0,70


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X=train_df[['word_count', 'any_neg', 'is_question', 'char_count']]
X_test_data = test_df[['word_count', 'any_neg', 'is_question', 'char_count']]
y=train_df.label
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.1,random_state=27)

In [25]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model = model.fit(Xtrain,ytrain)

In [27]:
ypred=model.predict(Xtest)

In [28]:
predict_test_value = model.predict(X_test_data)

In [29]:
predict_test_value.dtype

dtype('int64')

In [39]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RFR = RandomForestRegressor(
    n_estimators=100,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)

In [30]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(ytest,ypred)*100

76.76767676767676

In [34]:
submission = pd.read_csv("sample_submission_LnhVWA4.csv")

In [35]:
submission['id'] = test_df['id']
submission['label'] = predict_test_value

In [36]:
pd.DataFrame(submission, columns=['id', 'label']).to_csv('logistic.csv')